In [1]:
%run ../../NoteBooks/HocParse.ipynb
# This sets data

Section soma has 21 points!
Section axon has 172 points!
Section dend has 17 points!
	 132 more sections...


In [2]:
import asyncio
import websockets
import json

from neuron import h
from neuron.units import ms, mV

geometry_data = { "message" : "GEO_DATA", "segments": data }

async def handler(websocket):
    while True:
        try:
            data = await websocket.recv()
            data = json.loads(data)

            request_type = data["message"]

            match request_type:
                case "INIT":
                    response = json.dumps([json.dumps(geometry_data)])
                    print("\tSending data..")
                    await websocket.send(response)
                    break

                case "RUN_SIM":
                    print(data);
                    h.load_file("C:/Users/Jacob/Neuron-Web-Interface/Data/DeitcherEtAl2017/Morphs_and_trees/1148.hoc")
                    h.load_file('stdrun.hoc')

                    stim = h.IClamp(h.axon(1))
                    stim.delay = 5
                    stim.dur = 1
                    stim.amp = 0.1

                    v = h.Vector().record(h.soma(0.5)._ref_v)
                    t = h.Vector().record(h._ref_t)

                    h.finitialize(-65 * mV)
                    h.continuerun(25 * ms)
                    
                    print("DONE")
                    print(t)
                    print(v)
                    print(type(t))
                    break

        except websockets.exceptions.ConnectionClosedOK: 
            continue

        except json.decoder.JSONDecodeError:
            print("Invalid message:", data)
            break
       
async def main():
    async with websockets.serve(handler, "", 8001):
        await asyncio.Future()  # run forever

print("Running server:")
await main()
print("Done!")

Running server:
